PCA Jittering最早是由Alex在他2012年赢得的ImageNet竞赛的那篇NIPS中提出的，首先按照RGB三个颜色通道计算均值和标准差，对网络的输入数据进行规范化，随后我们在整个训练集上计算了协方差矩阵，进行特征分解，得到特征向量和特征值，用来做PCA Jittering。

参考: https://blog.csdn.net/l_xyy/article/details/71516071

* Collar jittering 色彩抖动
* PCA jittering

In [52]:
import numpy as np  
import os  
from PIL import Image
import argparse  
import random  
from scipy import misc
import matplotlib.pyplot as plt
  
def PCA_Jittering(path):  
    img_list = os.listdir(path)  
    img_num = len(img_list)  
      
    for i in range(img_num):  
        img_path = os.path.join(path, img_list[i])
        if img_list[i] == '.DS_Store':
            print("lalala")
        else:
            img = Image.open(img_path)  

            img = np.asanyarray(img, dtype = 'float32')  

            img = img // 255.0  
            img_size = img.size // 3  
            img1 = img.reshape(img_size, 3)  
            img1 = np.transpose(img1)  
            img_cov = np.cov([img1[0], img1[1], img1[2]])  
            lamda, p = np.linalg.eig(img_cov)  

            p = np.transpose(p)  

            alpha1 = random.normalvariate(0,0.3)  
            alpha2 = random.normalvariate(0,0.3)  
            alpha3 = random.normalvariate(0,0.3)  

            v = np.transpose((alpha1*lamda[0], alpha2*lamda[1], alpha3*lamda[2]))      
            add_num = np.dot(p,v)  

            img2 = np.array([img[:,:,0]+add_num[0], img[:,:,1]+add_num[1], img[:,:,2]+add_num[2]])  

            img2 = np.swapaxes(img2,0,2)  
            img2 = np.swapaxes(img2,0,1)  
            save_name = 'pre'+img_list[i]+'.png'  
            save_path = os.path.join(path, save_name)  
            misc.imsave(save_path,img2)  

#             plt.imshow(img2)  
#             plt.show()

In [53]:
PCA_Jittering('./train')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


lalala


In [59]:
"""数据增强
   1. 翻转变换 flip
   2. 随机修剪 random crop
   3. 色彩抖动 color jittering
   4. 平移变换 shift
   5. 尺度变换 scale
   6. 对比度变换 contrast
   7. 噪声扰动 noise
   8. 旋转变换/反射变换 Rotation/reflection
"""

from PIL import Image, ImageEnhance, ImageOps, ImageFile
import numpy as np
import random
import threading, os, time
import logging

logger = logging.getLogger(__name__)
ImageFile.LOAD_TRUNCATED_IMAGES = True


class DataAugmentation:
    """
    包含数据增强的八种方式
    """


    def __init__(self):
        pass

    @staticmethod
    def openImage(image):
        return Image.open(image, mode="r")

    @staticmethod
    def randomRotation(image, mode=Image.BICUBIC):
        """
         对图像进行随机任意角度(0~360度)旋转
        :param mode 邻近插值,双线性插值,双三次B样条插值(default)
        :param image PIL的图像image
        :return: 旋转转之后的图像
        """
        random_angle = np.random.randint(1, 360)
        return image.rotate(random_angle, mode)

    @staticmethod
    def randomCrop(image):
        """
        对图像随意剪切,考虑到图像大小范围(68,68),使用一个一个大于(36*36)的窗口进行截图
        :param image: PIL的图像image
        :return: 剪切之后的图像

        """
        image_width = image.size[0]
        image_height = image.size[1]
        crop_win_size = np.random.randint(40, 68)
        random_region = (
            (image_width - crop_win_size) >> 1, (image_height - crop_win_size) >> 1, (image_width + crop_win_size) >> 1,
            (image_height + crop_win_size) >> 1)
        return image.crop(random_region)

    @staticmethod
    def randomColor(image):
        """
        对图像进行颜色抖动
        :param image: PIL的图像image
        :return: 有颜色色差的图像image
        """
        random_factor = np.random.randint(0, 31) / 10.  # 随机因子
        color_image = ImageEnhance.Color(image).enhance(random_factor)  # 调整图像的饱和度
        random_factor = np.random.randint(10, 21) / 10.  # 随机因子
        brightness_image = ImageEnhance.Brightness(color_image).enhance(random_factor)  # 调整图像的亮度
        random_factor = np.random.randint(10, 21) / 10.  # 随机因1子
        contrast_image = ImageEnhance.Contrast(brightness_image).enhance(random_factor)  # 调整图像对比度
        random_factor = np.random.randint(0, 31) / 10.  # 随机因子
        return ImageEnhance.Sharpness(contrast_image).enhance(random_factor)  # 调整图像锐度

    @staticmethod
    def randomGaussian(image, mean=0.2, sigma=0.3):
        """
         对图像进行高斯噪声处理
        :param image:
        :return:
        """

        def gaussianNoisy(im, mean=0.2, sigma=0.3):
            """
            对图像做高斯噪音处理
            :param im: 单通道图像
            :param mean: 偏移量
            :param sigma: 标准差
            :return:
            """
            for _i in range(len(im)):
                im[_i] += random.gauss(mean, sigma)
            return im

        # 将图像转化成数组
        img = np.asarray(image)
        img.flags.writeable = True  # 将数组改为读写模式
        width, height = img.shape[:2]
        img_r = gaussianNoisy(img[:, :, 0].flatten(), mean, sigma)
        img_g = gaussianNoisy(img[:, :, 1].flatten(), mean, sigma)
        img_b = gaussianNoisy(img[:, :, 2].flatten(), mean, sigma)
        img[:, :, 0] = img_r.reshape([width, height])
        img[:, :, 1] = img_g.reshape([width, height])
        img[:, :, 2] = img_b.reshape([width, height])
        return Image.fromarray(np.uint8(img))

    @staticmethod
    def saveImage(image, path):
        image.save(path)


def makeDir(path):
    try:
        if not os.path.exists(path):
            if not os.path.isfile(path):
                # os.mkdir(path)
                os.makedirs(path)
            return 0
        else:
            return 1
    except Exception as e:
        print(str(e))
        return -2


def imageOps(func_name, image, des_path, file_name, times=5):
    funcMap = {"randomRotation": DataAugmentation.randomRotation,
               "randomCrop": DataAugmentation.randomCrop,
               "randomColor": DataAugmentation.randomColor,
               "randomGaussian": DataAugmentation.randomGaussian
               }
    if funcMap.get(func_name) is None:
        logger.error("%s is not exist", func_name)
        return -1

    for _i in range(0, times, 1):
        new_image = funcMap[func_name](image)
        DataAugmentation.saveImage(new_image, os.path.join(des_path, func_name + str(_i) + file_name))


opsList = {"randomRotation", "randomCrop", "randomColor", "randomGaussian"}


def threadOPS(path, new_path):
    """
    多线程处理事务
    :param src_path: 资源文件
    :param des_path: 目的地文件
    :return:
    """
    if os.path.isdir(path):
        img_names = os.listdir(path)
    else:
        img_names = [path]
    for img_name in img_names:
        print(img_name)
        if img_name == '.DS_Store':
            pass
        else:
            tmp_img_name = os.path.join(path, img_name)
            if os.path.isdir(tmp_img_name):
                if makeDir(os.path.join(new_path, img_name)) != -1:
                    threadOPS(tmp_img_name, os.path.join(new_path, img_name))
                else:
                    print('create new dir failure')
                    return -1
                    # os.removedirs(tmp_img_name)
            elif tmp_img_name.split('.')[1] != "DS_Store":
                # 读取文件并进行操作
                image = DataAugmentation.openImage(tmp_img_name)
                threadImage = [0] * 5
                _index = 0
                for ops_name in opsList:
                    threadImage[_index] = threading.Thread(target=imageOps,
                                                           args=(ops_name, image, new_path, img_name,))
                    threadImage[_index].start()
                    _index += 1
                    time.sleep(0.2)


if __name__ == '__main__':
    threadOPS("./train",
              "./preprocessing")

0a5bfa22ec15c6cb2fa63a9fe201a0f6.jpg
.DS_Store
0a6efa71252420d0f585dcc45ad86614.jpg
0a9d4f0a36078b271a1228dec8d41067.jpg
00a4af80b14c1c80fc0d973ea17b7c33.jpg
0a3b9ad1c11e859eac3599fcd4a4d34d.jpg
00a1cd3a42c5fa185c8bf7fa2b7da88e.jpg
0a7a64034ae5e64e618c18f88474eef8.jpg
DAMA.png


Exception in thread Thread-36:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-59-3931549042dc>", line 140, in imageOps
    new_image = funcMap[func_name](image)
  File "<ipython-input-59-3931549042dc>", line 60, in randomCrop
    return image.crop(random_region)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/PIL/Image.py", line 1044, in crop
    self.load()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/PIL/ImageFile.py", line 212, in load
    s = read(self.decodermaxblock)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 590, in load_read
    self.fp.read(

0a1ffe23e38528f18193c5a20475b92d.jpg
0a3b34ccaced672cf8481425d9f809de.jpg
0a8fe7c78a15a14b869d17a5db95769d.jpg


# 色彩抖动

In [ ]:
from PIL import Image, ImageEnhance, ImageOps, ImageFile

In [ ]:
import cv2
import numpy as np
iamge = 'image.jpg'
Image.open(image, mode="r")

In [68]:
rootdir = './train/collar_design_labels'  # 指明被遍历的文件夹
m = 0
outdir = './preprocessing/'

In [69]:
def randomColor(image):
    """
    对图像进行颜色抖动
    :param image: PIL的图像image
    :return: 有颜色色差的图像image
    """
    random_factor = np.random.randint(0, 31) / 10.  # 随机因子
    color_image = ImageEnhance.Color(image).enhance(random_factor)  # 调整图像的饱和度
    random_factor = np.random.randint(10, 21) / 10.  # 随机因子
    brightness_image = ImageEnhance.Brightness(color_image).enhance(random_factor)  # 调整图像的亮度
    random_factor = np.random.randint(10, 21) / 10.  # 随机因1子
    contrast_image = ImageEnhance.Contrast(brightness_image).enhance(random_factor)  # 调整图像对比度
    random_factor = np.random.randint(0, 31) / 10.  # 随机因子
    return ImageEnhance.Sharpness(contrast_image).enhance(random_factor)  # 调整图像锐度

In [70]:
for parent, dirnames, filenames in os.walk(rootdir):
    for filename in filenames:
        currentPath = os.path.join(parent, filename)
        img = Image.open(currentPath, mode="r")
        dst = randomColor(img)
        
        dst.save(outdir + filename + "-0.jpg")